
# Euron Week 6 — 회귀(LinearRegression/다항회귀) 과제

데이터셋: 
  - `fetch_california_housing()` (보스턴 데이터셋 사용 금지)
  - 합성 코사인 데이터 (문항 C에서 생성)

## 파트 구성
- **A. LinearRegression & OLS**
- **B. 회귀 평가 지표 & Scikit-Learn Scoring API**
- **C. 다항 회귀 & 과소적합/과적합 (Bias–Variance)**

> 유의: `load_boston()`는 삭제된 데이터셋입니다. **반드시** `fetch_california_housing()`을 사용하세요.



## A. LinearRegression & OLS

### 캘리포니아 주택 가격 예측 (fit_intercept 비교) — ★★
1) `fetch_california_housing()`으로 데이터셋(X, y)을 불러오고, `train_test_split(test_size=0.2, random_state=42)`로 분리하세요.  
2) `LinearRegression(fit_intercept=True)`과 `LinearRegression(fit_intercept=False)`를 각각 학습하고, 테스트 세트 **R²**를 비교하세요.  
3) 두 설정의 차이가 나는 이유를 **간단히** 서술하세요.


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
data = fetch_california_housing()
print(data)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n-

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 1) 데이터 로드 & 분리
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2) 두 모델 학습
lr_t = LinearRegression(fit_intercept=True)
lr_f = LinearRegression(fit_intercept=False)
lr_t.fit(X_train, y_train)
lr_f.fit(X_train, y_train)

# 3) 성능 비교
r2_t = r2_score(y_test, lr_t.predict(X_test))
r2_f = r2_score(y_test, lr_f.predict(X_test))
print("R2 (fit_intercept=True):", r2_t)
print("R2 (fit_intercept=False):", r2_f)

R2 (fit_intercept=True): 0.5757877060324526
R2 (fit_intercept=False): 0.5196561679229008



3. 두 설정의 차이가 나는 이유를 간단히 서술하세요.


fit_intercept=False를 하면 회귀 모델이 원점을 반드시 통과하도록 되므로 오차가 커져 R2가 낮아진다


## B. 회귀 평가 지표 & Scikit-Learn Scoring API (총 2문항)

### B-1. 지표 함수 구현 & 비교 — ★★
`mean_absolute_error, mean_squared_error, r2_score`를 이용해 MAE, MSE, RMSE, R²를 구현/출력하세요.  

In [4]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error
import numpy as np

y_true = y_test
y_pred = lr_t.predict(X_test)

mae = mean_absolute_error(y_test, lr_t.predict(X_test))
mse = mean_squared_error(y_test, lr_t.predict(X_test))
rmse = np.sqrt(mse)
r2 = r2_score(y_test, lr_t.predict(X_test))
print(mae, mse, rmse, r2)

0.5332001304956983 0.555891598695242 0.7455813830127748 0.5757877060324526



### B-2. `neg_mean_squared_error`의 의미 — ★
1) `cross_val_score(..., scoring='neg_mean_squared_error')`로 5-Fold 결과를 얻고, **실제 MSE**로 변환해 평균/표준편차를 출력하세요.  
2) 왜 `neg_` 접두사가 붙는지 한 줄로 설명하세요.


In [5]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(LinearRegression(), X, y, cv=5, scoring='neg_mean_squared_error')
real_mse = -scores

print(real_mse.mean(), real_mse.std())

0.5582901717686808 0.06560199778303905


B-2-2. 왜 neg_ 접두사가 붙는지 한 줄로 설명하세요.


neg_mean_squared_error는 MSE에 마이너스를 붙인 값이므로 실제 MSE를 얻기 위해 다시 마이너스를 붙임


## C. 다항 회귀 & 과소적합/과적합

### 코사인 데이터: Degree 1/4/15 비교 
1) 구간 `[0, 1]`에서 **30개**의 X를 임의 샘플링하고, `y = cos(2πX) + N(0, 0.1)`로 타깃을 생성하세요.  
2) `Pipeline([PolynomialFeatures(degree=d), LinearRegression()])`로 **degree=1,4,15** 모델을 학습하세요.  
3) 각 모델에 대해 **10-Fold MSE**를 구해 평균을 출력하고, **Underfitting/Good Fit/Overfitting**을 판별하세요.  


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score

rng = np.random.RandomState(0)
Xc = rng.rand(30, 1)
yc = np.cos(2*np.pi*Xc).ravel() + rng.normal(0, 0.1, size=30)

def cv_mse_for_degree(d):
    model = Pipeline([('poly', PolynomialFeatures(degree=d, include_bias=True)),
                      ('lr', LinearRegression())])
    scores = cross_val_score(model, Xc, yc, cv=KFold(n_splits=10, shuffle=True, random_state=0),
                             scoring='neg_mean_squared_error')
    return -scores.mean()

for d in [1,4,15]:     
    print(d, cv_mse_for_degree(d))

1 0.7226999447492073
4 0.020019218900056472
15 1290.3345849168654


degree 1 -> underfitting  
degree 4 -> good fit  
degree 15 -> overfitting